In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf

from fastDamerauLevenshtein import damerauLevenshtein
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from tensorflow.keras import layers

In [3]:
df = pd.read_csv('data/single_step_df_ints_2022-06-03_encoded.csv', header=0)

In [33]:
df_new[:1435]

,input,target,start_coords.x,start_coords.y,start_coords.z,p.already_seen,n.already_seen,w.already_seen,t.already_seen,g.already_seen,...,v.food_k,v.strong_k,v.mid_k,coordinates_y.x,coordinates_y.y,coordinates_y.z,y.containment,y.food_k,y.strong_k,y.mid_k
0,<start>,p,-0.451354,-0.413918,0.156247,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1,p,o,0.513000,-0.531000,0.740000,1.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
2,o,c,0.513000,-0.531000,0.740000,1.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
3,c,g,0.513000,-0.531000,0.740000,1.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
4,g,k,0.513000,-0.531000,0.740000,1.0,0.0,0.0,0.0,1.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1430,<start>,h,1.000000,3.000000,2.000000,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1431,h,h,0.000000,4.000000,1.000000,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1432,h,s,0.000000,4.000000,1.000000,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1433,s,d,0.000000,2.000000,2.000000,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0


In [15]:
float_cols = df.select_dtypes(include=['float64']).columns
df.loc[:, float_cols] = df.loc[:, float_cols].fillna(-99)

In [32]:
df['target'].unique()
df_new[:1435]['target'].value_counts()
#df[:1437]['target'].loc[df[:1437]['target'] == 'i'].index[0]

p    188
k    162
g    160
f    157
c    148
s    134
b     77
w     74
o     73
a     72
r     47
d     27
h     27
e     22
t     22
z     21
m     13
n      9
x      2
Name: target, dtype: int64

In [17]:
index_to_drop = df['target'].loc[df['target'] == 'y'].index[0]
index_to_drop

1945

In [18]:
nr_of_sequences = len(df[df['input'] == '<start>'])

In [28]:
# remove row with target that only occurs once

df_new = df.drop(index=[index_to_drop, 727], axis=0)
df_new.reset_index(inplace=True, drop=True)

In [34]:
# generate stratified split

split = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)

for train_index, test_index in split.split(df_new[:1435], df_new[:1435]['target']):
    strat_train = df_new.loc[train_index]
    strat_test_val = df_new.loc[test_index]

In [35]:
# split test_val into test and val

split_test_val = len(strat_test_val)//2

strat_test = strat_test_val[:split_test_val]
strat_val = strat_test_val[split_test_val:]

In [36]:
# generate list of labels to pass to MultiLabelBinarizer so there's the same number of
# classes for all datasets

labels = df_new['target'].unique()

In [37]:
len(labels)

23

In [38]:
def create_dataset(dataframe, labels, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels_ds = df.pop('target')
    encoder = MultiLabelBinarizer(classes=labels)
    encoded_labels = encoder.fit_transform(labels_ds)
    
    df = {key: value[:, tf.newaxis] for key, value in df.items()}
    dataset = tf.data.Dataset.from_tensor_slices((dict(df), encoded_labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(dataframe))
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(batch_size)
    
    return dataset

In [39]:
batch_size = 128

In [40]:
train_ds = create_dataset(strat_train, labels, batch_size=batch_size)
val_ds = create_dataset(strat_val, labels, shuffle=False, batch_size=batch_size)
test_ds = create_dataset(strat_test, labels, shuffle=False, batch_size=batch_size)

/tmp/ipykernel_10257/1510453973.py:7: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:, tf.newaxis] for key, value in df.items()}
2022-06-07 10:01:33.321333: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-07 10:01:33.477273: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-07 10:01:33.477866: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-07 10:01:33.481808: I tensorflow/core/platform/c

In [41]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
    normalizer = layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
    normalizer.adapt(feature_ds)
    
    return normalizer

In [42]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    if dtype == 'string':
        index = layers.StringLookup(max_tokens=max_tokens)
    else:
        index = layers.IntegerLookup(max_tokens=max_tokens)
        
    # prepare tf.data.Dataset that only yields the feature    
    feature_ds = dataset.map(lambda x, y: x[name])
    
    # learn set of possible values and assign fixed int index
    index.adapt(feature_ds)
    
    # encode int indices
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())
    
    # apply multi-hot encoding to indices
    # lambda function captures the layer to include them in Keras functional models later
    return lambda feature: encoder(index(feature))

In [43]:
def create_input_data(dataframe):
    all_inputs = []
    encoded_features = []
    
    for header in dataframe.columns:
        # numerical features
        if 'coord' in header or 'already' in header:
            numeric_col = tf.keras.Input(shape=(1,), name=header)
            normalization_layer = get_normalization_layer(header, train_ds)
            encoded_numeric_col = normalization_layer(numeric_col)
            all_inputs.append(numeric_col)
            encoded_features.append(encoded_numeric_col)
        
        # categorical features
        elif 'containment' in header or 'food' in header or 'mid' in header or \
        'strong' in header:
            categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='int64')
            encoding_layer = get_category_encoding_layer(name=header,
                                                        dataset=train_ds,
                                                        dtype='int64')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
        elif header == 'input':
            categorical_col = tf.keras.Input(shape=(1,), name='input', dtype='string')
            encoding_layer = get_category_encoding_layer(name='input',
                                                        dataset=train_ds,
                                                        dtype='string')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
    return all_inputs, encoded_features

In [44]:
all_inputs, encoded_features = create_input_data(df)

In [45]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

In [46]:
# use model from tutorial with dense layers

all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(512, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(len(labels))(x)

model = tf.keras.Model(all_inputs, output)

In [47]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             metrics=["accuracy"])

In [81]:
#tf.keras.utils.plot_model(model, show_shapes=True, rankdir='LR')

In [48]:
model.fit(train_ds, epochs=100, validation_data=val_ds, callbacks=[callback])

Epoch 1/100
8/8 [==============================] - 7s 304ms/step - loss: 3.0609 - accuracy: 0.1016 - val_loss: 10951.2148 - val_accuracy: 0.1620
Epoch 2/100
8/8 [==============================] - 0s 35ms/step - loss: 2.4901 - accuracy: 0.1763 - val_loss: 10627.0107 - val_accuracy: 0.2639
Epoch 3/100
8/8 [==============================] - 0s 35ms/step - loss: 2.2458 - accuracy: 0.2301 - val_loss: 11148.8018 - val_accuracy: 0.3056
Epoch 4/100
8/8 [==============================] - 0s 35ms/step - loss: 2.1061 - accuracy: 0.2560 - val_loss: 11253.6377 - val_accuracy: 0.3704
Epoch 5/100
8/8 [==============================] - 0s 35ms/step - loss: 2.0322 - accuracy: 0.2799 - val_loss: 11714.4424 - val_accuracy: 0.4074
Epoch 6/100
8/8 [==============================] - 0s 35ms/step - loss: 1.9708 - accuracy: 0.2928 - val_loss: 12037.5859 - val_accuracy: 0.3889
Epoch 7/100
8/8 [==============================] - 0s 36ms/step - loss: 1.8996 - accuracy: 0.3287 - val_loss: 13421.5254 - val_accuracy

8/8 [==============================] - 0s 35ms/step - loss: 1.0107 - accuracy: 0.6404 - val_loss: 23849.1699 - val_accuracy: 0.5602
Epoch 58/100
8/8 [==============================] - 0s 35ms/step - loss: 0.9961 - accuracy: 0.6584 - val_loss: 23373.7656 - val_accuracy: 0.5741
Epoch 59/100
8/8 [==============================] - 0s 36ms/step - loss: 0.9546 - accuracy: 0.6763 - val_loss: 23932.2129 - val_accuracy: 0.5648
Epoch 60/100
8/8 [==============================] - 0s 36ms/step - loss: 0.9838 - accuracy: 0.6624 - val_loss: 23806.2637 - val_accuracy: 0.5648
Epoch 61/100
8/8 [==============================] - 0s 36ms/step - loss: 0.9636 - accuracy: 0.6624 - val_loss: 23935.3633 - val_accuracy: 0.5463
Epoch 62/100
8/8 [==============================] - 0s 36ms/step - loss: 0.9698 - accuracy: 0.6504 - val_loss: 23981.5293 - val_accuracy: 0.5602
Epoch 63/100
8/8 [==============================] - 0s 36ms/step - loss: 0.9475 - accuracy: 0.6534 - val_loss: 22846.0176 - val_accuracy: 0.569

In [49]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

2/2 [==============================] - 0s 23ms/step - loss: 26811.5742 - accuracy: 0.5070
Accuracy 0.5069767236709595


In [50]:
model.save('models/next_obj_classifier_tablesetting_2022-06-07')

2022-06-07 10:03:41.886830: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/next_obj_classifier_tablesetting_2022-06-07/assets


INFO:tensorflow:Assets written to: models/next_obj_classifier_tablesetting_2022-06-07/assets


In [51]:
reloaded_model = tf.keras.models.load_model('models/next_obj_classifier_tablesetting_2022-06-07/')

In [ ]:
# test prediction for one sample

In [27]:
sample = df.loc[0].drop('target').to_dict()

In [28]:
input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}

In [29]:
prediction = reloaded_model.predict(input_dict)
prediction = tf.nn.sigmoid(prediction[0])

In [30]:
# get label for prediction

pred_label = labels[np.argmax(prediction)]
pred_label

'p'

In [52]:
def get_prequential_error(dataframe, model, labels, nr_of_sequences):
    errors = [[] for seq in range(0, nr_of_sequences + 1)]
    start_token_count = 0
    sequence_nr = 0
    
    for row in range(dataframe.index[0], dataframe.index[-1]): 
        observed_target = dataframe.loc[row, 'target']
        sample = dataframe.loc[row].drop('target').to_dict()
        input_dict = {name: tf.convert_to_tensor([value]) for name, value in 
                          sample.items()}
        predicted_target = model.predict(input_dict)
        predicted_target = tf.nn.sigmoid(predicted_target[0])
            
        pred_label = labels[np.argmax(predicted_target)]
        error = 1 - damerauLevenshtein(pred_label, observed_target)
        errors[sequence_nr].append(error)
        
        if row != 0 and dataframe.loc[row, 'input'] == '<start>':
            start_token_count += 1
        
        if start_token_count > 0:
            sequence_nr += 1
            start_token_count = 0
            
    return errors

In [53]:
df[1438:].index[-1]

2073

In [54]:
errors = get_prequential_error(df, reloaded_model, labels, nr_of_sequences)

In [55]:
summed_error = [sum(error) for error in errors[:-1]]

In [56]:
np.median(summed_error)

3.0

In [57]:
with open('results/nn_spatialinfo_prequential_summed_fitted_on_ts_results_all_2022-06-07.txt', 'w') as file:
    file.write(str(summed_error))

In [58]:
np.mean(summed_error)

3.466019417475728